In [61]:
import numpy as np
import pandas as pd
from math import *
from numpy.linalg import inv,det

In [62]:
constraints=pd.read_excel('constraints.xlsx')[1:]
constraints

,H_i,Unnamed: 1,B_j,Unnamed: 3,Unnamed: 4,B_i,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,1,aphid,NaN,NaN,NaN,0,NaN,NaN,t*pi,pi,0
2,2,ax,NaN,NaN,NaN,0,0 0,NaN,0,NaN,NaN
3,3,ay,NaN,NaN,NaN,0,0 0,NaN,0,NaN,NaN
4,4,r,1,0 0,NaN,0,2 0,NaN,NaN,NaN,NaN
5,5,t,2,0 0,1 0,1,0 0,1 0,NaN,NaN,NaN
6,6,ax,NaN,NaN,NaN,2,0 0,NaN,0,NaN,NaN
7,7,ay,NaN,NaN,NaN,2,0 0,NaN,-2,NaN,NaN


In [60]:
# for index, row in constraints.iterrows():
#     print(index)
#     print()
#     print(row)
#     print()
q0 = np.mat(np.random.random((3,1)))
q0

matrix([[0.91728373],
        [0.24351136],
        [0.1244833 ]])

In [45]:
def phiq_aphid(q,t,row,n):
    i=row['B_i']
    thisPHIq = np.mat([0,0,1])
    if i>0:
        thisPHIql = np.mat([0]*i*3)
        thisPHIq = np.hstack((thisPHIql,thisPHIq))

    if i*3+3<n :
        thisPHIqr = np.mat([0]*(n-i*3-3))
        thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

def phiq_ax(q,row,n):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    
    x=np.mat([1,0]).T
    R=np.mat([[0,-1],[1,0]])
    
    thisPHIq = x.T*R*Ai*si
    thisPHIq = np.hstack((x.T,thisPHIq))
    
    if i>0:
        thisPHIql = np.mat([0]*i*3)
        thisPHIq = np.hstack((thisPHIql,thisPHIq))

    if i*3+3<n :
        thisPHIqr = np.mat([0]*(n-i*3-3))
        thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

def phiq_ay(q,row,n):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    
    y=np.mat([0,1]).T
    R=np.mat([[0,-1],[1,0]])
    
    thisPHIq = y.T*R*Ai*si
    thisPHIq = np.hstack((y.T,thisPHIq))
    
    if i>0:
        thisPHIql = np.mat([0]*i*3)
        thisPHIq = np.hstack((thisPHIql,thisPHIq))

    if i*3+3<n :
        thisPHIqr = np.mat([0]*(n-i*3-3))
        thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

def phiq_rt(q,row,n):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    
    j=row['B_j']
    xj=q[j*3,0]
    yj=q[j*3+1,0]
    phij=q[j*3+2,0]
    rj=np.mat([xj,yj]).T
    Aj=np.mat([[cos(phij),-sin(phij)],[sin(phij),cos(phij)]])
    numbers = row['Unnamed: 3'].split(' ')
    numbers = [ float(x) for x in numbers ]
    sj=np.mat(numbers).T

    numbers = row['Unnamed: 7'].split(' ')
    numbers = [ float(x) for x in numbers ]
    v=np.mat(numbers).T
    c=float(row['Unnamed: 8'])
    R=np.mat([[0,-1],[1,0]])
    Bi=R*Ai
    Aij=Ai.T*Aj

    PHIqi=np.hstack(((-1)*v.T*Bi.T,(-1)*v.T*Ai.T*(rj-ri)-v.T*Aij*sj))/np.linalg.norm(v)
    PHIqj=np.hstack((v.T*Bi.T,v.T*Aij*sj))/np.linalg.norm(v)

    if i<j:
        if i+1==j:
            thisPHIq = np.hstack((PHIqi,PHIqj))
        else:
            thisPHIq = np.hstack((PHIqi,np.mat([0]*(j-i-1)*6).reshape(2,-1),PHIqj))
        
        if i>0:
            thisPHIql = np.mat([0]*i*6).reshape(2,-1)
            thisPHIq = np.hstack((thisPHIql,thisPHIq))

        if j*3+3<n :
            thisPHIqr = np.mat([0]*2*(n-j*3-3)).reshape(1,-1)
            thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    else:
        if j+1==i:
            thisPHIq = np.hstack((PHIqj,PHIqi))
        else:
            thisPHIq = np.hstack((PHIqj,np.mat([0]*(i-j-1)*6).reshape(2,-1),PHIqi))
        
        if j>0:
            thisPHIql = np.mat([0]*j*3)
            thisPHIq = np.hstack((thisPHIql,thisPHIq))

        if i*3+3<n :
            thisPHIqr = np.mat([0]*2*(n-i*3-3))
            thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

In [46]:
def calculate_PHIq(q, t, constraints):
    PHIq = np.mat([0]*q.shape[0])

    di=constraints['B_i']
    dj=constraints['B_j']
    df=pd.concat([di,dj], ignore_index=True)
    n = df.value_counts().shape[0]*3

    for index, row in constraints.iterrows():
        if row['Unnamed: 1'] == 'r':
            thisPHIq = phiq_r(q,row,n)

        if row['Unnamed: 1']=='aphid':
            thisPHIq = phiq_aphid(q,t,row,n)

        if row['Unnamed: 1'] == 'ax':
            thisPHIq = phiq_ax(q,row,n)

        if row['Unnamed: 1']=='ay':
            thisPHIq = phiq_ay(q,row,n)

        if row['Unnamed: 1']=='rt':
            thisPHIq = phiq_rt(q,row,n)
        print(row['Unnamed: 1'])
        print('thisPHIq')
        print(thisPHIq)
        print('PHIq')
        print(PHIq)
#         PHIq=np.vstack((PHIq,thisPHIq))

    return PHIq[1:]
t=1
q=np.mat([0,0,pi,0,0,pi]).T

calculate_PHIq(q, t, constraints)

aphid
thisPHIq
[[0 0 1 0 0 0]]
PHIq
[[0 0 0 0 0 0]]
ax
thisPHIq
[[1. 0. 0. 0. 0. 0.]]
PHIq
[[0 0 0 0 0 0]]
ay
thisPHIq
[[0. 1. 0. 0. 0. 0.]]
PHIq
[[0 0 0 0 0 0]]
ax
thisPHIq
[[0. 0. 0. 1. 0. 0.]]
PHIq
[[0 0 0 0 0 0]]
ay
thisPHIq
[[0. 0. 0. 0. 1. 0.]]
PHIq
[[0 0 0 0 0 0]]
rt
thisPHIq
[[-1.2246468e-16 -1.0000000e+00  2.0000000e+00  1.2246468e-16
   1.0000000e+00 -2.0000000e+00]]
PHIq
[[0 0 0 0 0 0]]


matrix([], shape=(0, 6), dtype=int32)

In [32]:
np.mat([0]*1*6).reshape(2,-1)
np.hstack((np.mat([0]*1*6).reshape(2,-1),np.mat([0]*1*6).reshape(2,-1)))

matrix([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]])

In [44]:
def phi_ax(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    x=np.mat([1,0]).T
    return x.T*(ri+Ai*si)-ci

def phi_ay(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    y=np.mat([0,1]).T
    return y.T*(ri+Ai*si)-ci

def phi_rt(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    
    j=row['B_j']
    xj=q[j*3,0]
    yj=q[j*3+1,0]
    phij=q[j*3+2,0]
    rj=np.mat([xj,yj]).T
    Aj=np.mat([[cos(phij),-sin(phij)],[sin(phij),cos(phij)]])
    numbers = row['Unnamed: 3'].split(' ')
    numbers = [ float(x) for x in numbers ]
    sj=np.mat(numbers).T

    numbers = row['Unnamed: 7'].split(' ')
    numbers = [ float(x) for x in numbers ]
    v=np.mat(numbers).T
    
    c=float(row['Unnamed: 8'])
    R=np.mat([[0,-1],[1,0]])

    h=rj+Aj*sj-ri-Ai*si
    
    return (R*Ai*v).T*h/np.linalg.norm(v)-c

In [47]:
def calculate_PHI(q, t, constraints):
    PHI = np.mat([0])
    for index, row in constraints.iterrows():
#         if row['Unnamed: 1'] == 'r':
#             thisPHI = phi_r(q,row)
#         if row['Unnamed: 1']=='ay':
#             thisPHI = phi_ay(q,row)
#             PHI=np.vstack((PHI,thisPHI))
#         if row['Unnamed: 1'] == 'ax':
#             thisPHI = phi_ax(q,row)
#             PHI=np.vstack((PHI,thisPHI))
        if row['Unnamed: 1'] == 'rt':
            thisPHI = phi_rt(q,row)
            PHI=np.vstack((PHI,thisPHI))
    return PHI[1:]
t=1
q=np.mat([0,0,pi/6,0,-2,pi/3]).T
calculate_PHI(q, t, constraints)

matrix([[4.4408921e-16]])

In [17]:
def gamma_ax(q,dq,row):
    i=row['B_i']
    phii=q[i*3+2,0]
    dphii=dq[i*3+2,0]
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    x=np.mat([1,0]).T

    return x.T*Ai*si*dphii*dphii

def gamma_ay(q,dq,row):
    i=row['B_i']
    phii=q[i*3+2,0]
    dphii=dq[i*3+2,0]
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    y=np.mat([0,1]).T

    return y.T*Ai*si*dphii*dphii

def gamma_rt(q,dq,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    dxi=dq[i*3,0]
    dyi=dq[i*3+1,0]
    dphii=dq[i*3+2,0]
    ri=np.mat([xi,yi]).T
    dri=np.mat([dxi,dyi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    
    j=row['B_j']
    xj=q[j*3,0]
    yj=q[j*3+1,0]
    phij=q[j*3+2,0]
    dxj=dq[j*3,0]
    dyj=dq[j*3+1,0]
    dphij=dq[j*3+2,0]
    rj=np.mat([xj,yj]).T
    drj=np.mat([dxj,dyj]).T
    Aj=np.mat([[cos(phij),-sin(phij)],[sin(phij),cos(phij)]])
    numbers = row['Unnamed: 3'].split(' ')
    numbers = [ float(x) for x in numbers ]
    sj=np.mat(numbers).T

    numbers = row['Unnamed: 7'].split(' ')
    numbers = [ float(x) for x in numbers ]
    v=np.mat(numbers).T
    c=float(row['Unnamed: 8'])
    R=np.mat([[0,-1],[1,0]])
    Bi=R*Ai
    Bij=R*Ai.T*Aj
    item1=(-2)*v.T*Ai.T*(drj-dri)*dphii
    item2=(-1)*v.T*Bi.T*(rj-ri)*dphii*dphii
    item3=v.T*Bij*sj*(dphij*dphij-dphii*dphii)*(dphij*dphij-dphii*dphii)
    return (item1+item2+item3)*(-1/np.linalg.norm(v))

In [18]:
def calculate_Gamma(q, dq, t, constraints):
    Gamma = np.mat([0])
    for index, row in constraints.iterrows():
#         if row['Unnamed: 1'] == 'r':
#             thisGamma = gamma_r(q,dq,row)
            
#         if row['Unnamed: 1']=='aphid':
#             thisGamma = gamma_aphid(q,t,row)
        if row['Unnamed: 1']=='ax':
            thisGamma = gamma_ax(q,dq,row)
            Gamma=np.vstack((Gamma,thisGamma))
        if row['Unnamed: 1']=='ay':
            thisGamma = gamma_ay(q,dq,row)
            Gamma=np.vstack((Gamma,thisGamma))
        if row['Unnamed: 1']=='rt':
            thisGamma = gamma_rt(q,dq,row)
            Gamma=np.vstack((Gamma,thisGamma))

    return Gamma[1:]
t=1
q=np.mat([0,0,0,0,0,0]).T
dq=np.mat([0,0,0,0,0,0]).T
calculate_Gamma(q, dq, t, constraints)

matrix([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [-0.]])

In [41]:
def v_r(q,row):
    return np.mat([0,0]).T
def v_ax(q,row):
    return np.mat([0]).T
def v_ay(q,row):
    return np.mat([0]).T
def v_aphi(q,row):
    return np.mat([0]).T

In [44]:
def calculate_V(q, t, constraints):
    V = np.mat([0])
    for index, row in constraints.iterrows():
        if row['Unnamed: 1'] == 'ax':
            thisV = v_ax(q,row)
            V=np.vstack((V,thisV))
#         if row['Unnamed: 1']=='aphid':
#             thisV = v_aphid(q,t,row)
#             V=np.vstack((V,thisV))
        if row['Unnamed: 1'] == 'ay':
            thisV = v_ay(q,row)
            V=np.vstack((V,thisV))
    return V[1:]
t=1
q=np.mat([0,0,0]).T
dq=np.mat([0,0,1]).T
calculate_V(q, t, constraints)

matrix([[0],
        [0]])